# 03 BMRA

Run BMRA based on global responses.

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data_dir_1 = "01_outputs_2020"
data_dir_2 = "02_outputs_2020"

out_dir = "03_outputs_2020"
os.makedirs(out_dir, exist_ok = True)

## Load data

### Global responses

Assembled from global responses for modules and DPD global responses.

In [3]:
def assemble_R_global(pathway_rglobal_file, dpd_rglobal_file):
    """
    Assemble full R_global matrix
    """
    R_global_modules_df = pd.read_csv(pathway_rglobal_file, index_col = 0)

    perts = R_global_modules_df.columns.tolist()

    R_global_DPD_df = pd.read_csv(dpd_rglobal_file, index_col = 0).T

    # filter by perts list
    R_global_DPD_df = R_global_DPD_df[R_global_DPD_df.index.isin(perts)]

    # sort by perts list
    R_global_DPD_df["sort_col"] = R_global_DPD_df.index.map({val: i for i, val in enumerate(perts)})
    R_global_DPD_df = R_global_DPD_df.sort_values("sort_col")
    R_global_DPD_df = R_global_DPD_df.drop("sort_col", axis = 1)

    # transpose
    R_global_DPD_df = R_global_DPD_df.T

    R_global_df = pd.concat([R_global_modules_df, R_global_DPD_df])

    return R_global_df

In [4]:
R_global_df = assemble_R_global(
    pathway_rglobal_file = os.path.join(data_dir_2, "R_global_annotated.csv"),
    dpd_rglobal_file = os.path.join(data_dir_1, "R_global_DPDonly_annotated.csv"),
)

modules = R_global_df.index.tolist()
exp_ids = R_global_df.columns.tolist()

R_global = R_global_df.values
n_modules = R_global.shape[0]

print(R_global.shape)
display(R_global_df)

(13, 435)


,LJP007_HUVEC_24H:B19,LJP007_HUVEC_24H:B20,LJP007_HUVEC_24H:B21,LJP007_HUVEC_24H:B22,LJP007_HUVEC_24H:B23,LJP007_HUVEC_24H:B24,LJP007_HUVEC_24H:C01,LJP007_HUVEC_24H:C02,LJP007_HUVEC_24H:C03,LJP007_HUVEC_24H:C04,...,REP.B011_HUVEC.A_24H:O09_2,REP.B011_HUVEC.A_24H:O10_2,REP.B011_HUVEC.A_24H:O11_2,REP.B011_HUVEC.A_24H:O12_2,siCDK2_STAT_vs_CTL_STAT,siCDK2_STAT_vs_CTL_STAT.1,siCDK2_STAT_vs_CTL_STAT.2,siCDK2_PSS_vs_CTL_STAT_ADJ_CTL_PSS_vs_CTL_STAT,siCDK2_PSS_vs_CTL_STAT_ADJ_CTL_PSS_vs_CTL_STAT.1,siCDK2_PSS_vs_CTL_STAT_ADJ_CTL_PSS_vs_CTL_STAT.2
CDK1,-0.241295,-0.049477,-0.118508,-0.064121,-0.059576,-0.085875,-0.034793,0.063252,0.001644,-0.064251,...,0.071982,-0.000142,-0.066307,-0.405803,-0.020968,-0.020968,-0.020968,-0.035646,-0.035646,-0.035646
CDK2,0.086828,0.034993,0.000712,-0.136315,-0.037666,0.006159,-0.139798,-0.084356,-0.065997,0.148656,...,-0.079839,-0.065996,-0.010269,-0.002830,-1.383500,-1.383500,-1.383500,-1.692456,-1.692456,-1.692456
PAK,-0.204719,0.053047,-0.012458,-0.235998,-0.010762,0.074992,-0.605308,-0.008513,0.148121,-0.031170,...,-0.048521,0.014415,-0.082089,0.115057,-0.042846,-0.042846,-0.042846,-0.081844,-0.081844,-0.081844
PI3K,-1.698457,-1.133142,-0.800476,-0.439644,-0.241075,-0.051966,-0.684202,-0.026276,-0.009121,-0.090468,...,-0.353995,0.063328,-0.437550,-0.038424,-0.013391,-0.013391,-0.013391,-0.286830,-0.286830,-0.286830
ERK,-0.173479,0.240846,-0.067897,-0.044627,-0.114116,0.103609,-1.983190,-0.149826,-0.041433,0.102618,...,-0.241006,0.176456,0.295115,0.127864,-0.199221,-0.199221,-0.199221,-0.325842,-0.325842,-0.325842
BET,-0.201354,-0.009742,-0.359697,-0.087660,0.423612,0.091018,0.334342,-0.014117,-0.171522,0.027898,...,-0.150613,0.537878,0.402017,0.372687,0.145809,0.145809,0.145809,0.516540,0.516540,0.516540
Aurora,-0.174423,0.115198,0.103587,0.379479,0.236328,-0.329926,-0.370444,-0.110243,-0.154645,-0.312533,...,-0.423675,0.058188,0.479511,-0.215884,0.006080,0.006080,0.006080,0.005779,0.005779,0.005779
TGFbR,0.048681,-0.100195,0.085931,0.108692,0.182509,0.090444,0.473756,0.222745,-0.086762,-0.109110,...,0.099206,0.064085,0.075046,0.090124,0.207455,0.207455,0.207455,0.189289,0.189289,0.189289
VEGFR,-1.102487,-0.044102,-0.567581,-0.286735,-0.374989,0.315450,-1.971104,-1.261839,-1.493498,-1.690930,...,0.047710,0.087880,0.342904,0.034504,-0.179015,-0.179015,-0.179015,-0.798205,-0.798205,-0.798205
PKC,0.043441,-0.023725,0.010902,0.015498,0.006574,0.017204,-0.007446,-0.012668,-0.004032,0.025198,...,0.003333,0.008737,0.001421,0.021584,0.000052,0.000052,0.000052,-0.070781,-0.070781,-0.070781


In [5]:
print(R_global_df.values.min())
print(R_global_df.values.max())

-8.262945823836553
9.271936346611776


### Perturbation matrix

In [6]:
pert_df = pd.read_csv(os.path.join(data_dir_1, "pert_annotated.csv"), index_col = 0)
display(pert_df)

pert_modules = pert_df.values

n_DPDs = R_global.shape[0] - pert_modules.shape[0]
pert_DPD = np.zeros((n_DPDs, len(exp_ids)))

pert = np.vstack([pert_modules, pert_DPD])

print(pert.shape)
print(pert)

,LJP007_HUVEC_24H:B19,LJP007_HUVEC_24H:B20,LJP007_HUVEC_24H:B21,LJP007_HUVEC_24H:B22,LJP007_HUVEC_24H:B23,LJP007_HUVEC_24H:B24,LJP007_HUVEC_24H:C01,LJP007_HUVEC_24H:C02,LJP007_HUVEC_24H:C03,LJP007_HUVEC_24H:C04,...,REP.B011_HUVEC.A_24H:O09_2,REP.B011_HUVEC.A_24H:O10_2,REP.B011_HUVEC.A_24H:O11_2,REP.B011_HUVEC.A_24H:O12_2,siCDK2_STAT_vs_CTL_STAT,siCDK2_STAT_vs_CTL_STAT.1,siCDK2_STAT_vs_CTL_STAT.2,siCDK2_PSS_vs_CTL_STAT_ADJ_CTL_PSS_vs_CTL_STAT,siCDK2_PSS_vs_CTL_STAT_ADJ_CTL_PSS_vs_CTL_STAT.1,siCDK2_PSS_vs_CTL_STAT_ADJ_CTL_PSS_vs_CTL_STAT.2
CDK1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CDK2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
PAK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PI3K,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ERK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BET,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Aurora,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TGFbR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
VEGFR,0,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
PKC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(13, 435)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### Networks

In [7]:
# empty prior
G_prior = np.zeros((n_modules, n_modules))

In [8]:
G_not = np.eye(n_modules)
G_not[:, n_modules - n_DPDs :] = 1

G_not

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.]])

## Run BMRA

In [9]:
n_gibbs = 200_000
n_window = 5_000

mode = "all"
n_iterations = 1

In [10]:
# here you must run bmra matlab code load to python A_mean, A_sd, r_mean, r_sd dataframes (i.e. BMRA output)


,CDK1,CDK2,PAK,PI3K,ERK,BET,Aurora,TGFbR,VEGFR,PKC,remod,FSS,OSS
CDK1,0.000000,0.982696,1.000000,1.000000,0.045228,0.060177,0.019754,0.000181,0.013147,0.000093,0.0,0.0,0.0
CDK2,1.000000,0.000000,1.000000,0.014223,0.115957,0.000541,0.005681,1.000000,0.981819,0.000000,0.0,0.0,0.0
PAK,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.640160,0.751011,0.000000,0.0,0.0,0.0
PI3K,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
ERK,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.0,0.0,0.0
BET,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.128483,0.0,0.0,0.0
Aurora,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.388115,0.067086,0.000000,0.0,0.0,0.0
TGFbR,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.200857,0.000000,1.000000,0.000000,0.0,0.0,0.0
VEGFR,1.000000,0.000000,0.087997,0.000000,1.000000,1.000000,0.401780,1.000000,0.000000,1.000000,0.0,0.0,0.0
PKC,0.005324,0.003971,0.002587,0.002481,0.011643,0.247647,0.833433,0.001827,0.033748,0.000000,0.0,0.0,0.0


,CDK1,CDK2,PAK,PI3K,ERK,BET,Aurora,TGFbR,VEGFR,PKC,remod,FSS,OSS
CDK1,0.000000e+00,1.304016e-01,6.661338e-16,6.661338e-16,2.078029e-01,2.378151e-01,1.391540e-01,1.344398e-02,1.139054e-01,9.652383e-03,0.0,0.0,0.0
CDK2,9.992007e-16,0.000000e+00,9.992007e-16,1.184109e-01,3.201731e-01,2.325117e-02,7.515537e-02,9.992007e-16,1.336066e-01,0.000000e+00,0.0,0.0,0.0
PAK,1.332268e-15,0.000000e+00,0.000000e+00,1.332268e-15,1.332268e-15,1.332268e-15,1.332268e-15,4.799533e-01,4.324273e-01,0.000000e+00,0.0,0.0,0.0
PI3K,3.219647e-15,0.000000e+00,3.219647e-15,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0
ERK,0.000000e+00,0.000000e+00,1.998401e-15,1.998401e-15,0.000000e+00,1.998401e-15,1.998401e-15,0.000000e+00,1.998401e-15,1.998401e-15,0.0,0.0,0.0
BET,1.332268e-15,0.000000e+00,0.000000e+00,0.000000e+00,1.332268e-15,0.000000e+00,0.000000e+00,1.332268e-15,1.332268e-15,3.346270e-01,0.0,0.0,0.0
Aurora,2.220446e-16,0.000000e+00,2.220446e-16,0.000000e+00,2.220446e-16,0.000000e+00,0.000000e+00,4.873210e-01,2.501714e-01,0.000000e+00,0.0,0.0,0.0
TGFbR,5.551115e-16,5.551115e-16,5.551115e-16,0.000000e+00,0.000000e+00,5.551115e-16,4.006415e-01,0.000000e+00,5.551115e-16,0.000000e+00,0.0,0.0,0.0
VEGFR,3.663736e-15,0.000000e+00,2.832913e-01,0.000000e+00,3.663736e-15,3.663736e-15,4.902579e-01,3.663736e-15,0.000000e+00,3.663736e-15,0.0,0.0,0.0
PKC,7.277154e-02,6.289040e-02,5.079581e-02,4.974378e-02,1.072726e-01,4.316455e-01,3.725889e-01,4.270650e-02,1.805809e-01,0.000000e+00,0.0,0.0,0.0


,CDK1,CDK2,PAK,PI3K,ERK,BET,Aurora,TGFbR,VEGFR,PKC,remod,FSS,OSS
CDK1,0.000000,-0.074474,0.221033,0.059186,-0.001109,-0.001765,0.000495,-0.000006,0.000206,0.000007,0.0,0.0,0.0
CDK2,-0.238574,0.000000,0.109963,0.000516,0.003985,0.000017,0.000149,-0.140548,0.048605,0.000000,0.0,0.0,0.0
PAK,0.415678,0.000000,0.000000,0.067023,0.192039,0.110830,0.133245,-0.077484,0.038841,0.000000,0.0,0.0,0.0
PI3K,0.235391,0.000000,0.191830,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
ERK,0.000000,0.000000,0.268371,0.199400,0.000000,0.118671,0.204413,0.000000,0.179537,0.519654,0.0,0.0,0.0
BET,-0.217233,0.000000,0.000000,0.000000,-0.094599,0.000000,0.000000,0.386160,-0.087427,0.038674,0.0,0.0,0.0
Aurora,-0.281734,0.000000,0.384997,0.000000,0.186631,0.000000,0.000000,-0.061037,0.004035,0.000000,0.0,0.0,0.0
TGFbR,0.352103,-0.102260,-0.077695,0.000000,0.000000,0.153122,-0.005942,0.000000,-0.111877,0.000000,0.0,0.0,0.0
VEGFR,0.252506,0.000000,0.011076,0.000000,0.261532,-0.210843,0.030968,-0.353630,0.000000,0.565831,0.0,0.0,0.0
PKC,-0.000050,-0.000034,0.000015,0.000010,0.000092,0.003580,0.010778,-0.000027,-0.000333,0.000000,0.0,0.0,0.0


,CDK1,CDK2,PAK,PI3K,ERK,BET,Aurora,TGFbR,VEGFR,PKC,remod,FSS,OSS
CDK1,0.000000,0.021650,0.014399,0.012908,0.005635,0.007597,0.003877,0.000435,0.002210,0.000682,0.0,0.0,0.0
CDK2,0.015530,0.000000,0.015049,0.004465,0.011530,0.000734,0.002065,0.018680,0.011199,0.000000,0.0,0.0,0.0
PAK,0.030716,0.000000,0.000000,0.014342,0.012846,0.016432,0.012172,0.062806,0.025368,0.000000,0.0,0.0,0.0
PI3K,0.005438,0.000000,0.004820,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
ERK,0.000000,0.000000,0.028614,0.025035,0.000000,0.023345,0.019825,0.000000,0.020398,0.084194,0.0,0.0,0.0
BET,0.017095,0.000000,0.000000,0.000000,0.009818,0.000000,0.000000,0.024081,0.010867,0.102059,0.0,0.0,0.0
Aurora,0.032941,0.000000,0.017476,0.000000,0.012858,0.000000,0.000000,0.078321,0.015288,0.000000,0.0,0.0,0.0
TGFbR,0.013158,0.013836,0.013972,0.000000,0.000000,0.008343,0.012508,0.000000,0.007058,0.000000,0.0,0.0,0.0
VEGFR,0.032594,0.000000,0.036316,0.000000,0.021869,0.019329,0.039519,0.034747,0.000000,0.066771,0.0,0.0,0.0
PKC,0.000782,0.000758,0.000347,0.000292,0.000954,0.006535,0.005990,0.000661,0.001941,0.000000,0.0,0.0,0.0


In [11]:
np.fill_diagonal(r_mean_df.values, -1)
rm_minus_inv = pd.DataFrame(np.linalg.pinv(r_mean_df), index = r_mean_df.index, columns = r_mean_df.columns)*(-1)
rm_minus_inv.to_csv(os.path.join(out_dir, "r_minv.csv"))
display(rm_minus_inv)

,CDK1,CDK2,PAK,PI3K,ERK,BET,Aurora,TGFbR,VEGFR,PKC,remod,FSS,OSS
CDK1,1.149525,-0.084772,0.305537,0.102254,0.069129,0.035058,0.056533,-0.008209,0.018460,0.047732,-3.018419e-16,-2.185752e-16,-2.081668e-16
CDK2,-0.254154,1.039306,0.101706,0.003446,0.055844,-0.030745,0.029799,-0.199301,0.089519,0.078482,3.577867e-16,4.271757e-16,2.602085e-18
PAK,0.492859,-0.027049,1.329009,0.180931,0.314438,0.147858,0.247891,-0.094501,0.105429,0.228776,-1.734723e-16,-1.110223e-16,-1.665335e-16
PI3K,0.365133,-0.025143,0.326864,1.058778,0.076591,0.036616,0.060860,-0.020060,0.024570,0.055122,-2.775558e-17,1.179612e-16,-3.885781e-16
ERK,0.208031,-0.006014,0.571662,0.294405,1.222539,0.145972,0.342541,-0.092955,0.240206,0.776860,-4.857226e-17,-2.914335e-16,-1.110223e-16
BET,-0.161065,-0.040006,-0.176575,-0.061480,-0.201053,1.075452,-0.075680,0.508573,-0.196101,-0.173847,-3.469447e-18,-3.608225e-16,3.330669e-16
Aurora,-0.114959,0.021310,0.538008,0.096411,0.336724,0.061381,1.146657,-0.124747,0.095503,0.231392,-3.469447e-17,-5.620504e-16,-1.665335e-16
TGFbR,0.339448,-0.144408,-0.072539,-0.002036,-0.086202,0.195463,-0.040868,1.164996,-0.172788,-0.135003,-1.040834e-17,-4.250073e-16,-2.289835e-16
VEGFR,0.259395,0.036910,0.323816,0.121948,0.425604,-0.242702,0.179349,-0.550147,1.174036,0.876088,1.734723e-16,1.665335e-16,-4.579670e-16
PKC,-0.001931,0.000046,0.005118,0.000813,0.002883,0.004603,0.012062,0.000625,-0.000039,1.001654,7.112366e-17,-1.838807e-16,-1.804112e-16


In [12]:
con_mat = pd.DataFrame(columns = ["From", "To", "Strength"])
for con_to in r_mean_df.index:
    for con_from in r_mean_df.columns:
        if ((r_mean_df.loc[con_to, con_from] != 0) & (con_from != con_to)):
            row_df = pd.DataFrame([[con_from, con_to, r_mean_df.loc[con_to, con_from]]], columns = ["From", "To", "Strength"])
            if con_mat.empty:
                con_mat = row_df.copy()
            else:
                con_mat = pd.concat([con_mat, row_df], axis = 0, ignore_index = True)
          
con_mat.to_csv(os.path.join(out_dir, "r_net.txt"), sep = "\t", index = False)
display(con_mat)

,From,To,Strength
0,CDK2,CDK1,-0.074474
1,PAK,CDK1,0.221033
2,PI3K,CDK1,0.059186
3,ERK,CDK1,-0.001109
4,BET,CDK1,-0.001765
...,...,...,...
79,PAK,OSS,2.357561
80,PI3K,OSS,-0.002097
81,ERK,OSS,0.886032
82,Aurora,OSS,0.490539


In [13]:
# now the same but with filtered matrices
# threshold value
Athr = 0.5

rm_filt = r_mean_df[np.abs(A_mean_df) > Athr].fillna(0).copy() #CHECK diag
np.fill_diagonal(rm_filt.values, -1)
rm_filt.to_csv(os.path.join(out_dir, "r_mean_filt.csv"))
display(rm_filt)

rm_filt_minus_inv = pd.DataFrame(np.linalg.pinv(rm_filt), index = rm_filt.index, columns = rm_filt.columns)*(-1)
rm_filt_minus_inv.to_csv(os.path.join(out_dir, "r_minv_filt.csv"))
display(rm_minus_inv)

con_mat = pd.DataFrame(columns = ["From", "To", "Strength"])
for con_to in rm_filt.index:
    for con_from in rm_filt.columns:
        if ((rm_filt.loc[con_to, con_from] != 0) & (con_from != con_to)):
            row_df = pd.DataFrame([[con_from, con_to, rm_filt.loc[con_to,con_from]]], columns = ["From", "To", "Strength"])
            if con_mat.empty:
                con_mat = row_df.copy()
            else:
                con_mat = pd.concat([con_mat, row_df], axis = 0, ignore_index = True)

con_mat.to_csv(os.path.join(out_dir, "r_net_filt.txt"), sep = "\t", index = False)
display(con_mat)


,CDK1,CDK2,PAK,PI3K,ERK,BET,Aurora,TGFbR,VEGFR,PKC,remod,FSS,OSS
CDK1,-1.000000,-0.074474,0.221033,0.059186,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
CDK2,-0.238574,-1.000000,0.109963,0.000000,0.000000,0.000000,0.000000,-0.140548,0.048605,0.000000,0.0,0.0,0.0
PAK,0.415678,0.000000,-1.000000,0.067023,0.192039,0.110830,0.133245,-0.077484,0.038841,0.000000,0.0,0.0,0.0
PI3K,0.235391,0.000000,0.191830,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
ERK,0.000000,0.000000,0.268371,0.199400,-1.000000,0.118671,0.204413,0.000000,0.179537,0.519654,0.0,0.0,0.0
BET,-0.217233,0.000000,0.000000,0.000000,-0.094599,-1.000000,0.000000,0.386160,-0.087427,0.000000,0.0,0.0,0.0
Aurora,-0.281734,0.000000,0.384997,0.000000,0.186631,0.000000,-1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
TGFbR,0.352103,-0.102260,-0.077695,0.000000,0.000000,0.153122,0.000000,-1.000000,-0.111877,0.000000,0.0,0.0,0.0
VEGFR,0.252506,0.000000,0.000000,0.000000,0.261532,-0.210843,0.000000,-0.353630,-1.000000,0.565831,0.0,0.0,0.0
PKC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010778,0.000000,0.000000,-1.000000,0.0,0.0,0.0


,CDK1,CDK2,PAK,PI3K,ERK,BET,Aurora,TGFbR,VEGFR,PKC,remod,FSS,OSS
CDK1,1.149525,-0.084772,0.305537,0.102254,0.069129,0.035058,0.056533,-0.008209,0.018460,0.047732,-3.018419e-16,-2.185752e-16,-2.081668e-16
CDK2,-0.254154,1.039306,0.101706,0.003446,0.055844,-0.030745,0.029799,-0.199301,0.089519,0.078482,3.577867e-16,4.271757e-16,2.602085e-18
PAK,0.492859,-0.027049,1.329009,0.180931,0.314438,0.147858,0.247891,-0.094501,0.105429,0.228776,-1.734723e-16,-1.110223e-16,-1.665335e-16
PI3K,0.365133,-0.025143,0.326864,1.058778,0.076591,0.036616,0.060860,-0.020060,0.024570,0.055122,-2.775558e-17,1.179612e-16,-3.885781e-16
ERK,0.208031,-0.006014,0.571662,0.294405,1.222539,0.145972,0.342541,-0.092955,0.240206,0.776860,-4.857226e-17,-2.914335e-16,-1.110223e-16
BET,-0.161065,-0.040006,-0.176575,-0.061480,-0.201053,1.075452,-0.075680,0.508573,-0.196101,-0.173847,-3.469447e-18,-3.608225e-16,3.330669e-16
Aurora,-0.114959,0.021310,0.538008,0.096411,0.336724,0.061381,1.146657,-0.124747,0.095503,0.231392,-3.469447e-17,-5.620504e-16,-1.665335e-16
TGFbR,0.339448,-0.144408,-0.072539,-0.002036,-0.086202,0.195463,-0.040868,1.164996,-0.172788,-0.135003,-1.040834e-17,-4.250073e-16,-2.289835e-16
VEGFR,0.259395,0.036910,0.323816,0.121948,0.425604,-0.242702,0.179349,-0.550147,1.174036,0.876088,1.734723e-16,1.665335e-16,-4.579670e-16
PKC,-0.001931,0.000046,0.005118,0.000813,0.002883,0.004603,0.012062,0.000625,-0.000039,1.001654,7.112366e-17,-1.838807e-16,-1.804112e-16


,From,To,Strength
0,CDK2,CDK1,-0.074474
1,PAK,CDK1,0.221033
2,PI3K,CDK1,0.059186
3,CDK1,CDK2,-0.238574
4,PAK,CDK2,0.109963
5,TGFbR,CDK2,-0.140548
6,VEGFR,CDK2,0.048605
7,CDK1,PAK,0.415678
8,PI3K,PAK,0.067023
9,ERK,PAK,0.192039


In [14]:
# number of zero elements in the matrix
display(len(rm_filt.index)*len(rm_filt.columns) - rm_filt.astype(bool).sum().sum())

100

In [15]:
# number of non-zero elements in the matrix
display(rm_filt.astype(bool).sum().sum() - len(modules))

56